In [1]:
# Installing requeried libraries

%pip install yfinance==0.2.41
%pip install crewai==0.28.8
%pip install 'crewai[tools]'
%pip install langchain==0.1.20
%pip install langchain-openai==0.1.7
%pip install langchain-community==0.0.38
%pip install duckduckgo-search==5.3.0

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import datetime
from dotenv import load_dotenv

import yfinance as yf

from crewai import Agent, Task, Crew, Process

from langchain.tools import Tool
from langchain_openai import ChatOpenAI
from langchain_community.tools import DuckDuckGoSearchResults

from IPython.display import Markdown

In [3]:
# Loading API KEY from .env and defining the LLM model
load_dotenv()
llm = ChatOpenAI(model='gpt-3.5-turbo')

In [4]:
# Setting current date
curr_date = str(datetime.date.today())
last_year = str(datetime.date.today() - datetime.timedelta(weeks=52))

In [5]:
# Creating Tool to search stock prices for a ticket
def fetch_stock_price(ticket):
    stock = yf.download(ticket, start=last_year, end=curr_date)
    return stock

yahoo_finance_tool = Tool(
    name = 'Yahoo Finance Tool',
    description = 'Fetches stock prices for {ticket} from a specified date from Yahoo Finance API',
    func = lambda ticket: fetch_stock_price(ticket)
)

In [6]:
# Creating the Agent who will analyse the stock price
stockPriceAnalyst = Agent(
    role = 'Senior Stock Price Analyst',
    goal = 'Find the {ticket} stock price and analyses trends',
    backstory = 'You have extensive experience in analysing the price of a specified stock and making predictions about its future price.',
    llm = llm,
    verbose = True,
    max_iter = 5,
    memory = True,
    allow_delegation = False,
    tools = [yahoo_finance_tool]
)

In [7]:
# Creating stock price Task for the Agent
getStockPrice = Task(
    description = 'Analyse the {ticket} stock price history and create a trend analysis of up, down or sideways',
    expected_output = '''Specify the current trend stock price - up, down or sideways.
    eg. stock='AAPL, price up' ''',
    agent = stockPriceAnalyst
)

In [8]:
# Using the DuckDuckGo Tool to search news for a stock ticket company
search_tool = DuckDuckGoSearchResults(backend='news', num_results=10)

In [9]:
# Creating the Agent who will analyse the stock company news
newsAnalyst = Agent(
    role = 'Stock News Analyst',
    goal = '''Create a short summary of the market news related to the stock {ticket} company. Specify the
    current trend - up, down or sideways with the news context. For each request stock asset, specify a number between 0 and 100,
    where 0 is extreme fear and 100 is extreme greed.''',
    backstory = '''You're highly experienced in analysing the market trends and news and have trecked assets for more than 10 year.
    You're also a master level analyst in traditional markets and have deep understanding in human psychology.
    You understand news, their titles and information, but you look at those with a healthy dose of skepticism.
    You consider also the source of the news articles.''',
    llm = llm,
    verbose = True,
    max_iter = 10,
    memory = True,
    allow_delegation = False,
    tools = [search_tool]
)

In [10]:
# Creating stock news Task for the Agent
getNews = Task(
    description = f'''Take the stock and always include BTC to it (if not requested).
    Use the search tool to search each one individually.
    The current date is {curr_date}.
    Compose the results into a helpful report.''',
    expected_output = '''A summary of the overall market and a one sentence summary for each requested asset.
    Include a fear/greed score for each asset based on the news. Use the format:
    <STOCK ASSET>
    <SUMMARY BASED ON NEWS>
    <TREND PREDICTION>
    <FEAR/GREED SCORE>''',
    agent = newsAnalyst
    )

In [11]:
# Creating the Agent who will analyse the stock price
stockAnalystWriter = Agent(
    role = 'Senior Stock Analyst Writer',
    goal = 'Analyse the trends price and news and write an insightful and informative 3 paragraphs long newsletter based on the stock report and price trend.',
    backstory = '''You're widely accepted as the best stock analyst in the market. You understand complex concepts and create compelling
    stories and narratives that resonate with wider audiences.
    You understand macro factors and combine multiple theories - eg. cycle theory and fundamental analyses.
    You're able to hold multiple opinions when analysing anything.''',
    llm = llm,
    verbose = True,
    max_iter = 5,
    memory = True,
    allow_delegation = True,
)

In [12]:
# Creating stock news Task for the Agent
writeAnalysis = Task(
    description = '''Use the stock price trend and the stock news report to create an analysis and write the newsletter
    about the {ticket} company that is brief and highlight the most import points.
    Focus in the stock price trend, news and fear/greed score. What are the near future considerations?
    Include the previous analysis of stock trend and news summary.''',
    expected_output = '''An eloquent 3 paragraphs newsletter formatted as markdown in an easy readable manner. It should contain:
    - 3 bullets executive summary
    - Introduction - set the overall picture and spike up the interest
    - Main part provides the meat of the analysis including the news summary and fear/greed scores
    - Summary - key facts and concrete future trend prediction - up, down or sideways.''',
    agent = stockAnalystWriter,
    context = [getStockPrice, getNews]
    )

In [13]:
# Creating Crew
crew = Crew(
    agents = [stockPriceAnalyst, newsAnalyst, stockAnalystWriter],
    tasks = [getStockPrice, getNews, writeAnalysis],
    verbose = True,
    process = Process.hierarchical,
    full_output = True,
    share_crew = False,
    manager_llm = llm,
    max_iter = 15
)

In [14]:
results = crew.kickoff(inputs={'ticket': 'AAPL'})

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Analyse the AAPL stock price history and create a trend analysis of up, down or sideways


> Entering new CrewAgentExecutor chain...
I need to gather information on the AAPL stock price history and determine the current trend to complete the task successfully.

Action: Delegate work to co-worker
Action Input: {
    "coworker": "Senior Stock Price Analyst",
    "task": "Analyse AAPL stock price history and determine current trend",
    "context": "I need you to analyze the historical stock price data of AAPL and provide a detailed trend analysis of whether the stock price is currently trending up, down, or sideways. Please consider all relevant factors and provide a thorough analysis."
}

> Entering new CrewAgentExecutor chain...
I need to analyze the historical stock price data of AAPL and determine the current trend based on all relevant factors.

Action: Yahoo Finance Tool
Action Input: {"ticket": "AAPL", "date": "01/01/20

[*********************100%%**********************]  1 of 1 completed


 

                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2023-08-22  177.059998  177.679993  176.250000  177.229996  176.327698   
2023-08-23  178.520004  181.550003  178.330002  181.119995  180.197906   
2023-08-24  180.669998  181.100006  176.009995  176.380005  175.482025   
2023-08-25  177.380005  179.149994  175.820007  178.610001  177.700684   
2023-08-28  180.089996  180.589996  178.550003  180.190002  179.272629   
...                ...         ...         ...         ...         ...   
2024-08-13  219.009995  221.889999  219.009995  221.270004  221.270004   
2024-08-14  220.570007  223.029999  219.699997  221.720001  221.720001   
2024-08-15  224.600006  225.350006  222.759995  224.720001  224.720001   
2024-08-16  223.919998  226.830002  223.649994  226.050003  226.050003   
2024-08-19  225.720001  225.990005  223.039993  225.889999  225.889999   

              Volume  
Date       

In [16]:
print(results.keys())

dict_keys(['final_output', 'tasks_outputs'])


In [17]:
Markdown(results['final_output'])

# AAPL Stock Analysis Newsletter

## Executive Summary:
- The AAPL stock price trend is currently up with fluctuations, indicating a positive trajectory.
- Fear/greed score for AAPL is neutral, suggesting cautious optimism among investors.
- Recent news events have caused some volatility in AAPL stock, with mixed sentiment from analysts.

## Introduction:
In the world of tech giants, AAPL stands out as a beacon of innovation and success. With its current upward trend in stock price and neutral fear/greed score, investors are cautiously optimistic about the future of the company. However, recent news events have introduced some volatility into the market, requiring a closer look at the factors influencing AAPL's performance.

## Main Analysis:
The recent decision by Warren Buffett to reduce his stake in AAPL has caused some uncertainty among investors, leading to fluctuations in the stock price. Despite this, analysts at Bank of America have maintained a Buy rating on AAPL with a price target of $256, indicating long-term confidence in the company's growth potential. It is crucial to monitor how these conflicting signals play out in the market and how they may impact AAPL's future performance.

## Summary:
In conclusion, while AAPL's stock price trend remains positive overall, recent news events have introduced some volatility into the market. The mixed sentiment from analysts underscores the need for a cautious approach to investing in AAPL. Looking ahead, it is crucial to pay close attention to how the company navigates these challenges and whether it can maintain its positive trajectory in the face of fluctuating market conditions.